## Preprocessing

In [38]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import numpy as np

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()
print(len(application_df)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [40]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [41]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [42]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [43]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [44]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts[classification_counts > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [45]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts.astype(int) < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [46]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [47]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Number of features after preprocessing: {X.shape[1]}")

Number of features after preprocessing: 43


In [48]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
tf.random.set_seed(42)
nn = tf.keras.models.Sequential()

# --------------------------------------------------------------------------------------------------------------
# NON-OPTIMIZED FIRST ATTEMPT: 72.33% PREDICTED ACCURACY
# --------------------------------------------------------------------------------------------------------------
# First hidden layer
# nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
# nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# --------------------------------------------------------------------------------------------------------------
# OPTIMIZATION ATTEMPT #1: 72.64% PREDICTED ACCURACY
# --------------------------------------------------------------------------------------------------------------
# First hidden layer
# nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
# nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# --------------------------------------------------------------------------------------------------------------
# OPTIMIZATION ATTEMPT 2: 72.68% PREDICTED ACCURACY
# --------------------------------------------------------------------------------------------------------------
# First hidden layer
# nn.add(Dense(units=128, activation='tanh', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
# nn.add(Dense(units=64, activation='tanh'))

# Third hidden layer
# nn.add(Dense(units=64, activation='tanh'))

# Fourth hidden layer
# nn.add(Dense(units=32, activation='tanh'))

# --------------------------------------------------------------------------------------------------------------
# OPTIMIZATION ATTEMPT 3: 72.26% PREDICTED ACCURACY
# --------------------------------------------------------------------------------------------------------------
# First hidden layer
# nn.add(Dense(units=50, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
# nn.add(Dense(units=256, activation='tanh'))

# Third hidden layer
# nn.add(Dense(units=25, activation='relu'))

# Fourth hidden layer
# nn.add(Dense(units=64, activation='tanh'))

# --------------------------------------------------------------------------------------------------------------
# OPTIMIZATION ATTEMPT 4: 72.59% PREDICTED ACCURACY
# --------------------------------------------------------------------------------------------------------------
# First hidden layer
nn.add(Dense(units=150, activation='tanh', input_dim=X_train_scaled.shape[1]))

# # Second hidden layer
nn.add(Dense(units=125, activation='relu'))

# # Third hidden layer
nn.add(Dense(units=110, activation='tanh'))

# Fourth hidden layer
nn.add(Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_57 (Dense)                     │ (None, 150)                 │           6,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_58 (Dense)                     │ (None, 125)                 │          18,875 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_59 (Dense)                     │ (None, 110)                 │          13,860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 64)                  │           7,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,504 (181.66 KB)

 Trainable params: 46,504 (181.66 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the callback to save weights every 5 epochs
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='model_weights_epoch_{epoch:02d}.weights.h5',
    save_weights_only=True,
    save_freq=5 * (100),
    verbose=1
)

In [89]:
# Train the model
# nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1)

X_train_scaled = np.array(X_train_scaled, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)


nn.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint_callback],
    validation_data=(X, y)
)


Epoch 1/100
481/858 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7121 - loss: 0.5794
Epoch 1: saving model to model_weights_epoch_01.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7172 - loss: 0.5742 - val_accuracy: 0.4676 - val_loss: 0.8113
Epoch 2/100
128/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7379 - loss: 0.5508
Epoch 2: saving model to model_weights_epoch_02.weights.h5
641/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7320 - loss: 0.5545
Epoch 2: saving model to model_weights_epoch_02.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7314 - loss: 0.5550 - val_accuracy: 0.4676 - val_loss: 0.7839
Epoch 3/100
267/858 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7347 - loss: 0.5498
Epoch 3: saving model to model_weights_epoch_03.weights.h5
777/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7334 - loss: 0.5510
Epoch 3: saving model to model_weights_epoch_03.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7332 

In [90]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 2ms/step - accuracy: 0.7259 - loss: 0.5736
Loss: 0.5736197233200073, Accuracy: 0.7259474992752075


In [91]:
  # Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization4.h5")

In [92]:
from google.colab import files
files.download('/content/AlphabetSoupCharity_Optimization4.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>